In [1]:
import requests
import pprint
from urllib.parse import urlencode, urlparse, parse_qsl

In [2]:
api_key = ""
with open('api_key.txt') as f:
    api_key = f.readline()
    f.close

##### Google Maps Client API

In [3]:
class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = 'json'
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception("API key is required")
        self.api_key = api_key
        self.location_query = address_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
    
    # returns the json of the object as nested dicts
    def json(self):
        loc_query = self.location_query
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
     # display the json of the object as nested dicts in a prettify way
    def show(self):
        loc_query = self.location_query
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299):
            print("error")
        pprint.pprint(r.json())    
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)
        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range(200, 299): 
            return {}
        latlng = {}
        try:
            latlng = r.json()['results'][0]['geometry']['location']
        except:
            pass
        lat,lng = latlng.get("lat"), latlng.get("lng")
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword="Mexican food", radius=5000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location=location)
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": radius,
            "keyword": keyword
        }
        params_encoded = urlencode(params)
        places_url = f"{endpoint}?{params_encoded}"
        r = requests.get(places_url)
        # print(places_url, r.text)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def detail(self, place_id="ChIJlXOKcDC3j4ARzal-5j-p-FY", fields=["name", "rating", "formatted_phone_number", "formatted_address"]):
        detail_base_endpoint = f"https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        detail_params = {
            "place_id": f"{place_id}",
            "fields" : ",".join(fields),
            "key": self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endpoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [4]:
client = GoogleMapsClient(api_key=api_key, address_or_postal_code='rabat')
print(client.lat, client.lng)

33.9715904 -6.8498129


In [5]:
client.search("Tacos", radius=5000)

{'html_attributions': [],
 'next_page_token': 'Aap_uED40dYtIwl_HHn2WFSK_gjF_Q9I3yo98wWJ30DhxA3Mm_24_nvGYjPlvl4TL0cADng-0cKLCPqSKqpJIGzFlZhwlJa37XoNV3gLm8KphuUDDfTypJq6-6E-YOVqhJUO-5I18iWg7rHlJa1o5jXj3fgc6PIRf-KEK4nSPmOEcBj_PQ8hqpbj3PK5qfBPbXPwNE35YZaJdugntfyEtHFwl4bZ6fvK4qdYVSkPW20zKAPpuwAmKMWWQIxmgdjXgRGXOA3kEHfabkWuB-u5z9rm63ojK9yd5LqO-K4PDtRLxbavSDd8Vl6nkEdyonXMzrhOFLek2qnGhEVPAgdSodC82pr5zMZa9IAE10vcghZ1HIHBV-ogPMC9aa_hBYqvPa3kVtLpA8azSZ_lTicdoUNafugF8R_2G7s5gi0d_NrWK_wUv42jyw5l4Q',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 33.9536237, 'lng': -6.8635094},
    'viewport': {'northeast': {'lat': 33.95484532989272,
      'lng': -6.862311120107278},
     'southwest': {'lat': 33.95214567010728, 'lng': -6.865010779892722}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'name': 'Tacos de Paris',
   'opening_hours': {'open_now': True},
   'place_id': 'ChIJd8pwjoUTpw0R4bEVI8iQ2o0',
   'plus_code': {'c

In [6]:
client.detail(place_id='ChIJRfptTwEg3YARACDUpVSiRso')

{'html_attributions': [],
 'result': {'formatted_address': '407 31st St, Newport Beach, CA 92663, USA',
  'formatted_phone_number': '(949) 673-9453',
  'name': 'Wild Taco',
  'rating': 4.4},
 'status': 'OK'}

Returning the json object as a nested dict

In [7]:
client.json()

{'results': [{'address_components': [{'long_name': 'Rabat',
     'short_name': 'Rabat',
     'types': ['locality', 'political']},
    {'long_name': 'Rabat',
     'short_name': 'Rabat',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Rabat-Salé-Kénitra',
     'short_name': 'Rabat-Salé-Kénitra',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Morocco',
     'short_name': 'MA',
     'types': ['country', 'political']}],
   'formatted_address': 'Rabat, Morocco',
   'geometry': {'bounds': {'northeast': {'lat': 34.0349463, 'lng': -6.791642},
     'southwest': {'lat': 33.9034914, 'lng': -6.9228836}},
    'location': {'lat': 33.9715904, 'lng': -6.8498129},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 34.0349463, 'lng': -6.791642},
     'southwest': {'lat': 33.9034914, 'lng': -6.9228836}}},
   'place_id': 'ChIJwcVQH4drpw0RawcIdO1GyXo',
   'types': ['locality', 'political']}],
 'status': 'OK'}

Display json result in a prettified manner

In [8]:
client.show()

{'results': [{'address_components': [{'long_name': 'Rabat',
                                      'short_name': 'Rabat',
                                      'types': ['locality', 'political']},
                                     {'long_name': 'Rabat',
                                      'short_name': 'Rabat',
                                      'types': ['administrative_area_level_2',
                                                'political']},
                                     {'long_name': 'Rabat-Salé-Kénitra',
                                      'short_name': 'Rabat-Salé-Kénitra',
                                      'types': ['administrative_area_level_1',
                                                'political']},
                                     {'long_name': 'Morocco',
                                      'short_name': 'MA',
                                      'types': ['country', 'political']}],
              'formatted_address': 'Rabat, Morocco',
    